# 태그 추출

In [1]:
import pandas as pd 
import numpy as np 
import koreanize_matplotlib
import matplotlib.pyplot as plt

In [2]:
review_df = pd.read_csv('/Users/ranny/LAB/review_df_stopwords.csv', encoding='utf-8')

In [3]:
review_df = review_df[['diner_idx', 'reviewer_id', 'notstopwords']]

In [4]:
from tqdm import tqdm

# TF-IDF 행렬 생성
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(review_df['notstopwords'])

# 단어 목록 가져오기
words = tfidf.get_feature_names_out()

# 각 리뷰에서 상위 5개 태그 추출하는 함수
def extract_top_n_tags(tfidf_matrix, words, top_n=5):
    tags_per_review = []
    
    for i in tqdm(range(tfidf_matrix.shape[0])):
        # 리뷰에 대한 TF-IDF 값을 얻기
        row = tfidf_matrix[i].toarray().flatten()
        
        # TF-IDF 값이 높은 상위 top_n 단어의 인덱스를 찾음
        top_n_indices = row.argsort()[-top_n:][::-1]
        
        # 해당 인덱스의 단어를 태그로 추출
        top_n_words = [words[idx] for idx in top_n_indices]
        tags_per_review.append(top_n_words)
    
    return tags_per_review

# 리뷰별 상위 5개 태그 추출
tags = extract_top_n_tags(tfidf_matrix, words, top_n=5)

# 태그를 데이터프레임에 추가
review_df['tags'] = tags

100%|███████████████████████████████████| 96573/96573 [01:03<00:00, 1520.95it/s]


In [5]:
review_df.head()

,diner_idx,reviewer_id,notstopwords,tags
0,216642463,정희,반찬 가짓수 맛 은 평범할 수 있는데 정도 면 깨끗하고 규모 커서 혼밥 도 무리 없어요,"[평범할, 무리, 규모, 가짓수, 커서]"
1,9346076,이지수,근처 운동 끝나고 서 제육 먹으면 그렇게 맛있을 수가 없습니다 반찬 밥 국 다 셀프...,"[제육, 원하는만큼, 운동, 보러, 끝나고]"
2,9346076,손석윤,일주일 한번 씩 은 꼭 먹는 코다리 찜 사장 님 직접 소스 비법 만들어서 먹고나면 ...,"[코다리, 생각나는구나, 토욜, 비법, 야근]"
3,26961859,세렌디피티,예전 맛있게 먹던 집 오랜 만 생각나서 갔더니 올 3월 더 영업 안 한다고 함,"[3월, 생각나서, 갔더니, 먹던, 한다고]"
4,26961859,고독한 탐식가,인당 일만원 이라는 어마어마한 가격 김치찌개 육질 좋은 삼겹살 사용 하고 정갈한 밑...,"[일만원, 어마어마한, 육질, 그렇다, 정갈한]"


In [6]:
review_df['tags'] = review_df['tags'].apply(lambda x: ' '.join(x))

In [7]:
review_df.head()

,diner_idx,reviewer_id,notstopwords,tags
0,216642463,정희,반찬 가짓수 맛 은 평범할 수 있는데 정도 면 깨끗하고 규모 커서 혼밥 도 무리 없어요,평범할 무리 규모 가짓수 커서
1,9346076,이지수,근처 운동 끝나고 서 제육 먹으면 그렇게 맛있을 수가 없습니다 반찬 밥 국 다 셀프...,제육 원하는만큼 운동 보러 끝나고
2,9346076,손석윤,일주일 한번 씩 은 꼭 먹는 코다리 찜 사장 님 직접 소스 비법 만들어서 먹고나면 ...,코다리 생각나는구나 토욜 비법 야근
3,26961859,세렌디피티,예전 맛있게 먹던 집 오랜 만 생각나서 갔더니 올 3월 더 영업 안 한다고 함,3월 생각나서 갔더니 먹던 한다고
4,26961859,고독한 탐식가,인당 일만원 이라는 어마어마한 가격 김치찌개 육질 좋은 삼겹살 사용 하고 정갈한 밑...,일만원 어마어마한 육질 그렇다 정갈한


In [8]:
review_df.to_csv('review_tag.csv', index=False)

# 분리 후 태그 추출

In [9]:
review_tag = pd.read_csv('/Users/ranny/Downloads/review_tag.csv')

In [10]:
# 20000개만 뽑아서 시도
review_tag= review_tag.sample(n=20000, random_state=42)

In [11]:
review_df.columns

Index(['diner_idx', 'reviewer_id', 'notstopwords', 'tags'], dtype='object')

In [12]:
review_df = review_df[['diner_idx', 'reviewer_id', 'notstopwords']]
review_df

,diner_idx,reviewer_id,notstopwords
0,216642463,정희,반찬 가짓수 맛 은 평범할 수 있는데 정도 면 깨끗하고 규모 커서 혼밥 도 무리 없어요
1,9346076,이지수,근처 운동 끝나고 서 제육 먹으면 그렇게 맛있을 수가 없습니다 반찬 밥 국 다 셀프...
2,9346076,손석윤,일주일 한번 씩 은 꼭 먹는 코다리 찜 사장 님 직접 소스 비법 만들어서 먹고나면 ...
3,26961859,세렌디피티,예전 맛있게 먹던 집 오랜 만 생각나서 갔더니 올 3월 더 영업 안 한다고 함
4,26961859,고독한 탐식가,인당 일만원 이라는 어마어마한 가격 김치찌개 육질 좋은 삼겹살 사용 하고 정갈한 밑...
...,...,...,...
96568,1068851079,블블,6시 오픈 이라길래 6시 반에 갔는데 웨이 팅 왜케 많아 ㅠㅠ 오리 지날 양 꼬치 ...
96569,1068851079,정은,리지 널 양 꼬치 데리야끼 양 꼬치
96570,1068851079,파루,너무 양 냄새 없다는 장점 이자 단점 양 갈비 강추 익히는데 걸리지만 그만큼 맛있다...
96571,1068851079,그리고,리지 널 양 갈비 180 g 13000원 순한 양 꼬치 180 g 12000원 가지...


In [13]:
pip install konlpy pandas


Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
from konlpy.tag import Okt


# Okt 객체 생성
okt = Okt()

# 명사와 기타 품사 분리 함수
def extract_nouns_and_others(text):
    # 형태소 분석
    pos_tags = okt.pos(text)

    # 명사와 기타 품사 분리
    nouns = [word for word, pos in pos_tags if pos in ['Noun']]
    others = [word for word, pos in pos_tags if pos not in ['Noun']]

    return ' '.join(nouns), ' '.join(others)

# 명사와 기타 품사 컬럼 추가
review_df[['nouns', 'others']] = review_df['notstopwords'].apply(extract_nouns_and_others).apply(pd.Series)

In [15]:
review_df

,diner_idx,reviewer_id,notstopwords,nouns,others
0,216642463,정희,반찬 가짓수 맛 은 평범할 수 있는데 정도 면 깨끗하고 규모 커서 혼밥 도 무리 없어요,반찬 가짓수 맛 은 수 정도 면 규모 커서 혼밥 도 무리,평범할 있는데 깨끗하고 없어요
1,9346076,이지수,근처 운동 끝나고 서 제육 먹으면 그렇게 맛있을 수가 없습니다 반찬 밥 국 다 셀프...,근처 운동 제육 수가 반찬 밥 국 셀프 담 수 수 제육 고기 도 진짜 메뉴 도 예정,끝나고 서 먹으면 그렇게 맛있을 없습니다 다 있어서 원하는만큼 먹을 있고 많이 주세...
2,9346076,손석윤,일주일 한번 씩 은 꼭 먹는 코다리 찜 사장 님 직접 소스 비법 만들어서 먹고나면 ...,일주일 한번 은 꼭 코다리 찜 사장 직접 소스 비법 집 맛집 이기 집 코다리 이집 ...,씩 먹는 님 만들어서 먹고나면 생각나는 먹는내내 생각나는구나 맛있는
3,26961859,세렌디피티,예전 맛있게 먹던 집 오랜 만 생각나서 갔더니 올 3월 더 영업 안 한다고 함,예전 집 만 더 영업 안 함,맛있게 먹던 오랜 생각나서 갔더니 올 3월 한다고
4,26961859,고독한 탐식가,인당 일만원 이라는 어마어마한 가격 김치찌개 육질 좋은 삼겹살 사용 하고 정갈한 밑...,일만원 가격 김치찌개 육질 삼겹살 사용 밑반찬 도 가성,인당 이라는 어마어마한 좋은 하고 정갈한 좋았지만 비는 그저 그렇다
...,...,...,...,...,...
96568,1068851079,블블,6시 오픈 이라길래 6시 반에 갔는데 웨이 팅 왜케 많아 ㅠㅠ 오리 지날 양 꼬치 ...,오픈 반 웨이 팅 왜케 오리 양 꼬치 데리야끼 양 꼬치 바 로우 칭따오 맥주 여자 ...,6시 이라길래 6시 에 갔는데 많아 ㅠㅠ 지날 꿔 먹다가 배불러 죽는줄 알았다 그치...
96569,1068851079,정은,리지 널 양 꼬치 데리야끼 양 꼬치,리지 널 양 꼬치 데리야끼 양 꼬치,
96570,1068851079,파루,너무 양 냄새 없다는 장점 이자 단점 양 갈비 강추 익히는데 걸리지만 그만큼 맛있다...,양 냄새 장점 이자 단점 양 갈비 강추 우도,너무 없다는 익히는데 걸리지만 그만큼 맛있다 꿔 특별했다
96571,1068851079,그리고,리지 널 양 갈비 180 g 13000원 순한 양 꼬치 180 g 12000원 가지...,리지 널 양 갈비 양 꼬치 가지 튀김 볶음 하얼빈 맥주 양 꼬치 해도 무방 정도 고...,180 g 13000원 순한 180 g 12000원 17000원 7000원 는 잡내...


In [16]:
# '음식'과 '진짜' 단어가 있으면 제외
excluded_words = ['음식', '진짜', '너무']
review_df['nouns'] = review_df['nouns'].apply(
    lambda x: ' '.join([noun for noun in x.split() if noun not in excluded_words])
)

excluded_words = ['음식', '진짜', '너무']
review_df['others'] = review_df['others'].apply(
    lambda x: ' '.join([others for others in x.split() if others not in excluded_words])
)
review_df

,diner_idx,reviewer_id,notstopwords,nouns,others
0,216642463,정희,반찬 가짓수 맛 은 평범할 수 있는데 정도 면 깨끗하고 규모 커서 혼밥 도 무리 없어요,반찬 가짓수 맛 은 수 정도 면 규모 커서 혼밥 도 무리,평범할 있는데 깨끗하고 없어요
1,9346076,이지수,근처 운동 끝나고 서 제육 먹으면 그렇게 맛있을 수가 없습니다 반찬 밥 국 다 셀프...,근처 운동 제육 수가 반찬 밥 국 셀프 담 수 수 제육 고기 도 메뉴 도 예정,끝나고 서 먹으면 그렇게 맛있을 없습니다 다 있어서 원하는만큼 먹을 있고 많이 주세...
2,9346076,손석윤,일주일 한번 씩 은 꼭 먹는 코다리 찜 사장 님 직접 소스 비법 만들어서 먹고나면 ...,일주일 한번 은 꼭 코다리 찜 사장 직접 소스 비법 집 맛집 이기 집 코다리 이집 ...,씩 먹는 님 만들어서 먹고나면 생각나는 먹는내내 생각나는구나 맛있는
3,26961859,세렌디피티,예전 맛있게 먹던 집 오랜 만 생각나서 갔더니 올 3월 더 영업 안 한다고 함,예전 집 만 더 영업 안 함,맛있게 먹던 오랜 생각나서 갔더니 올 3월 한다고
4,26961859,고독한 탐식가,인당 일만원 이라는 어마어마한 가격 김치찌개 육질 좋은 삼겹살 사용 하고 정갈한 밑...,일만원 가격 김치찌개 육질 삼겹살 사용 밑반찬 도 가성,인당 이라는 어마어마한 좋은 하고 정갈한 좋았지만 비는 그저 그렇다
...,...,...,...,...,...
96568,1068851079,블블,6시 오픈 이라길래 6시 반에 갔는데 웨이 팅 왜케 많아 ㅠㅠ 오리 지날 양 꼬치 ...,오픈 반 웨이 팅 왜케 오리 양 꼬치 데리야끼 양 꼬치 바 로우 칭따오 맥주 여자 ...,6시 이라길래 6시 에 갔는데 많아 ㅠㅠ 지날 꿔 먹다가 배불러 죽는줄 알았다 그치...
96569,1068851079,정은,리지 널 양 꼬치 데리야끼 양 꼬치,리지 널 양 꼬치 데리야끼 양 꼬치,
96570,1068851079,파루,너무 양 냄새 없다는 장점 이자 단점 양 갈비 강추 익히는데 걸리지만 그만큼 맛있다...,양 냄새 장점 이자 단점 양 갈비 강추 우도,없다는 익히는데 걸리지만 그만큼 맛있다 꿔 특별했다
96571,1068851079,그리고,리지 널 양 갈비 180 g 13000원 순한 양 꼬치 180 g 12000원 가지...,리지 널 양 갈비 양 꼬치 가지 튀김 볶음 하얼빈 맥주 양 꼬치 해도 무방 정도 고...,180 g 13000원 순한 180 g 12000원 17000원 7000원 는 잡내...


In [17]:
# TF-IDF 계산
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(review_df['nouns'])

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# 상위 2개 TF-IDF 값 추출하여 'n_keyword' 열에 추가
review_df['n_keyword'] = tfidf_df.apply(lambda row: row.nlargest(2).index.tolist(), axis=1)

In [18]:
review_df

,diner_idx,reviewer_id,notstopwords,nouns,others,n_keyword
0,216642463,정희,반찬 가짓수 맛 은 평범할 수 있는데 정도 면 깨끗하고 규모 커서 혼밥 도 무리 없어요,반찬 가짓수 맛 은 수 정도 면 규모 커서 혼밥 도 무리,평범할 있는데 깨끗하고 없어요,"[무리, 규모]"
1,9346076,이지수,근처 운동 끝나고 서 제육 먹으면 그렇게 맛있을 수가 없습니다 반찬 밥 국 다 셀프...,근처 운동 제육 수가 반찬 밥 국 셀프 담 수 수 제육 고기 도 메뉴 도 예정,끝나고 서 먹으면 그렇게 맛있을 없습니다 다 있어서 원하는만큼 먹을 있고 많이 주세...,"[제육, 운동]"
2,9346076,손석윤,일주일 한번 씩 은 꼭 먹는 코다리 찜 사장 님 직접 소스 비법 만들어서 먹고나면 ...,일주일 한번 은 꼭 코다리 찜 사장 직접 소스 비법 집 맛집 이기 집 코다리 이집 ...,씩 먹는 님 만들어서 먹고나면 생각나는 먹는내내 생각나는구나 맛있는,"[코다리, 토욜]"
3,26961859,세렌디피티,예전 맛있게 먹던 집 오랜 만 생각나서 갔더니 올 3월 더 영업 안 한다고 함,예전 집 만 더 영업 안 함,맛있게 먹던 오랜 생각나서 갔더니 올 3월 한다고,"[영업, 예전]"
4,26961859,고독한 탐식가,인당 일만원 이라는 어마어마한 가격 김치찌개 육질 좋은 삼겹살 사용 하고 정갈한 밑...,일만원 가격 김치찌개 육질 삼겹살 사용 밑반찬 도 가성,인당 이라는 어마어마한 좋은 하고 정갈한 좋았지만 비는 그저 그렇다,"[일만원, 육질]"
...,...,...,...,...,...,...
96568,1068851079,블블,6시 오픈 이라길래 6시 반에 갔는데 웨이 팅 왜케 많아 ㅠㅠ 오리 지날 양 꼬치 ...,오픈 반 웨이 팅 왜케 오리 양 꼬치 데리야끼 양 꼬치 바 로우 칭따오 맥주 여자 ...,6시 이라길래 6시 에 갔는데 많아 ㅠㅠ 지날 꿔 먹다가 배불러 죽는줄 알았다 그치...,"[꼬치, 묵음]"
96569,1068851079,정은,리지 널 양 꼬치 데리야끼 양 꼬치,리지 널 양 꼬치 데리야끼 양 꼬치,,"[꼬치, 데리야끼]"
96570,1068851079,파루,너무 양 냄새 없다는 장점 이자 단점 양 갈비 강추 익히는데 걸리지만 그만큼 맛있다...,양 냄새 장점 이자 단점 양 갈비 강추 우도,없다는 익히는데 걸리지만 그만큼 맛있다 꿔 특별했다,"[이자, 우도]"
96571,1068851079,그리고,리지 널 양 갈비 180 g 13000원 순한 양 꼬치 180 g 12000원 가지...,리지 널 양 갈비 양 꼬치 가지 튀김 볶음 하얼빈 맥주 양 꼬치 해도 무방 정도 고...,180 g 13000원 순한 180 g 12000원 17000원 7000원 는 잡내...,"[가지, 튀김]"


In [19]:
review_df.loc[1000]

diner_idx                              8846917
reviewer_id                   이명학(Michael Lee)
notstopwords    논골 집은 본점 이든 체인 이든 갈비살 맛 같아서 좋음
nouns                           논골 본점 체인 갈비살 맛
others                         집은 이든 이든 같아서 좋음
n_keyword                             [논골, 체인]
Name: 1000, dtype: object

In [20]:
review_df['good'] = review_df['others'].apply(lambda x: '맛있다' if '맛있' in x else None)
review_df

,diner_idx,reviewer_id,notstopwords,nouns,others,n_keyword,good
0,216642463,정희,반찬 가짓수 맛 은 평범할 수 있는데 정도 면 깨끗하고 규모 커서 혼밥 도 무리 없어요,반찬 가짓수 맛 은 수 정도 면 규모 커서 혼밥 도 무리,평범할 있는데 깨끗하고 없어요,"[무리, 규모]",None
1,9346076,이지수,근처 운동 끝나고 서 제육 먹으면 그렇게 맛있을 수가 없습니다 반찬 밥 국 다 셀프...,근처 운동 제육 수가 반찬 밥 국 셀프 담 수 수 제육 고기 도 메뉴 도 예정,끝나고 서 먹으면 그렇게 맛있을 없습니다 다 있어서 원하는만큼 먹을 있고 많이 주세...,"[제육, 운동]",맛있다
2,9346076,손석윤,일주일 한번 씩 은 꼭 먹는 코다리 찜 사장 님 직접 소스 비법 만들어서 먹고나면 ...,일주일 한번 은 꼭 코다리 찜 사장 직접 소스 비법 집 맛집 이기 집 코다리 이집 ...,씩 먹는 님 만들어서 먹고나면 생각나는 먹는내내 생각나는구나 맛있는,"[코다리, 토욜]",맛있다
3,26961859,세렌디피티,예전 맛있게 먹던 집 오랜 만 생각나서 갔더니 올 3월 더 영업 안 한다고 함,예전 집 만 더 영업 안 함,맛있게 먹던 오랜 생각나서 갔더니 올 3월 한다고,"[영업, 예전]",맛있다
4,26961859,고독한 탐식가,인당 일만원 이라는 어마어마한 가격 김치찌개 육질 좋은 삼겹살 사용 하고 정갈한 밑...,일만원 가격 김치찌개 육질 삼겹살 사용 밑반찬 도 가성,인당 이라는 어마어마한 좋은 하고 정갈한 좋았지만 비는 그저 그렇다,"[일만원, 육질]",None
...,...,...,...,...,...,...,...
96568,1068851079,블블,6시 오픈 이라길래 6시 반에 갔는데 웨이 팅 왜케 많아 ㅠㅠ 오리 지날 양 꼬치 ...,오픈 반 웨이 팅 왜케 오리 양 꼬치 데리야끼 양 꼬치 바 로우 칭따오 맥주 여자 ...,6시 이라길래 6시 에 갔는데 많아 ㅠㅠ 지날 꿔 먹다가 배불러 죽는줄 알았다 그치...,"[꼬치, 묵음]",None
96569,1068851079,정은,리지 널 양 꼬치 데리야끼 양 꼬치,리지 널 양 꼬치 데리야끼 양 꼬치,,"[꼬치, 데리야끼]",None
96570,1068851079,파루,너무 양 냄새 없다는 장점 이자 단점 양 갈비 강추 익히는데 걸리지만 그만큼 맛있다...,양 냄새 장점 이자 단점 양 갈비 강추 우도,없다는 익히는데 걸리지만 그만큼 맛있다 꿔 특별했다,"[이자, 우도]",맛있다
96571,1068851079,그리고,리지 널 양 갈비 180 g 13000원 순한 양 꼬치 180 g 12000원 가지...,리지 널 양 갈비 양 꼬치 가지 튀김 볶음 하얼빈 맥주 양 꼬치 해도 무방 정도 고...,180 g 13000원 순한 180 g 12000원 17000원 7000원 는 잡내...,"[가지, 튀김]",None


In [21]:
review_df['bad'] = review_df['others'].apply(lambda x: '맛없다' if '맛없' in x else None)

In [22]:
review_df[review_df['bad'].notnull()]

,diner_idx,reviewer_id,notstopwords,nouns,others,n_keyword,good,bad
170,1811240711,디공,사실 정말 맛 없습니다 김밥 그나 만 괜찮지만 분식 이렇게 맛없게 할수 있나 싶네요,사실 정말 맛 김밥 그나 만 분식,없습니다 괜찮지만 이렇게 맛없게 할수 있나 싶네요,"[그나, 분식]",None,맛없다
286,2134340448,고기튀김,고기 는 종이 같고 양배추 도 귀찮은지 생강 대체 방토 는 시들함 밥 도 맛없음 진...,고기 종이 양배추 도 생강 대체 방토 밥 도 집 더 보기,는 같고 귀찮은지 는 시들함 맛없음 맛있었던 인데 아쉽,"[방토, 생강]",맛있다,맛없다
303,829412960,리뷰,냉면 소고기 15 000원 일행 동성 맛없다는 평이 다 소고기 는 다 먹지도 않았다...,냉면 소고기 일행 동성 평이 소고기 앚 편이 고 간장 향 도 재 방문 의사 제로,15 000원 맛없다는 다 는 다 먹지도 않았다 짠 강했다 다,"[소고기, 동성]",None,맛없다
318,737445391,bbqm12,빵 커피 는 딱히 맛없지도 맛있지도 안았지만 사장 님 보이시 는 분 께서 너무 불친...,빵 커피 사장 보이시 분 느낌 빵 주 실수 못 빵집 빵 못 브랜드 지점 후기 본점 ...,는 딱히 맛없지도 맛있지도 안았지만 님 는 께서 불친절하다는 받았다 대워 없냐고 여...,"[후기, 보이시]",맛있다,맛없다
417,1942795823,김,붕어빵 은 괜찮은데 베이글 은 질겅질겅 너무 질 기고 맛없어요 빵 아니라 곱창 인 ...,붕어빵 은 베이글 은 질겅질겅 질 기고 빵 곱창 인 줄 입 베이글 레시피,괜찮은데 맛없어요 아니라 먹었는데 한 먹고 버림 바꾸셔야 할듯,"[베이글, 질겅질겅]",None,맛없다
...,...,...,...,...,...,...,...,...
96252,1555439101,유자,개 같은 맛 이에요 맛없어요 돈 아까 움,개 맛 돈 움,같은 이에요 맛없어요 아까,"[가가, 가감]",None,맛없다
96270,1555439101,구송현,개 맛없어요 맛있는지 궁금해서 방문 해봤는데 진심 맛 없었어요 말 그대로 그냥 개 ...,개 방문 진심 맛 말 그대로 그냥 개,맛없어요 맛있는지 궁금해서 해봤는데 없었어요 비추 입니다,"[진심, 그대로]",맛있다,맛없다
96299,1555439101,I could fly,맛없음,,맛없음,"[가가, 가감]",None,맛없다
96304,1555439101,모던항,맛없고 돈 개 깢 서비스 도 구림 데이트 개 망쳤내,돈 개 깢 서비스 도 구림 데이트 개,맛없고 망쳤내,"[구림, 데이트]",None,맛없다


In [23]:
review_df['others'] = review_df['others'].str.replace('맛있', '', regex=False)
review_df['others'] = review_df['others'].str.replace('맛없', '', regex=False)


# TF-IDF 계산
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(review_df['others'])

# TF-IDF 결과를 데이터프레임으로 변환
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# 상위 2개 TF-IDF 값 추출하여 'o_keyword' 열에 추가
review_df['o_keyword'] = tfidf_df.apply(lambda row: row.nlargest(3).index.tolist(), axis=1)


In [24]:
review_df['keyword'] = review_df['n_keyword'] + review_df['o_keyword']

In [25]:
review_df

,diner_idx,reviewer_id,notstopwords,nouns,others,n_keyword,good,bad,o_keyword,keyword
0,216642463,정희,반찬 가짓수 맛 은 평범할 수 있는데 정도 면 깨끗하고 규모 커서 혼밥 도 무리 없어요,반찬 가짓수 맛 은 수 정도 면 규모 커서 혼밥 도 무리,평범할 있는데 깨끗하고 없어요,"[무리, 규모]",None,None,"[평범할, 깨끗하고, 없어요]","[무리, 규모, 평범할, 깨끗하고, 없어요]"
1,9346076,이지수,근처 운동 끝나고 서 제육 먹으면 그렇게 맛있을 수가 없습니다 반찬 밥 국 다 셀프...,근처 운동 제육 수가 반찬 밥 국 셀프 담 수 수 제육 고기 도 메뉴 도 예정,끝나고 서 먹으면 그렇게 을 없습니다 다 있어서 원하는만큼 먹을 있고 많이 주세요 ...,"[제육, 운동]",맛있다,None,"[원하는만큼, 보러, 끝나고]","[제육, 운동, 원하는만큼, 보러, 끝나고]"
2,9346076,손석윤,일주일 한번 씩 은 꼭 먹는 코다리 찜 사장 님 직접 소스 비법 만들어서 먹고나면 ...,일주일 한번 은 꼭 코다리 찜 사장 직접 소스 비법 집 맛집 이기 집 코다리 이집 ...,씩 먹는 님 만들어서 먹고나면 생각나는 먹는내내 생각나는구나 는,"[코다리, 토욜]",맛있다,None,"[생각나는구나, 먹는내내, 먹고나면]","[코다리, 토욜, 생각나는구나, 먹는내내, 먹고나면]"
3,26961859,세렌디피티,예전 맛있게 먹던 집 오랜 만 생각나서 갔더니 올 3월 더 영업 안 한다고 함,예전 집 만 더 영업 안 함,게 먹던 오랜 생각나서 갔더니 올 3월 한다고,"[영업, 예전]",맛있다,None,"[3월, 생각나서, 갔더니]","[영업, 예전, 3월, 생각나서, 갔더니]"
4,26961859,고독한 탐식가,인당 일만원 이라는 어마어마한 가격 김치찌개 육질 좋은 삼겹살 사용 하고 정갈한 밑...,일만원 가격 김치찌개 육질 삼겹살 사용 밑반찬 도 가성,인당 이라는 어마어마한 좋은 하고 정갈한 좋았지만 비는 그저 그렇다,"[일만원, 육질]",None,None,"[어마어마한, 그렇다, 정갈한]","[일만원, 육질, 어마어마한, 그렇다, 정갈한]"
...,...,...,...,...,...,...,...,...,...,...
96568,1068851079,블블,6시 오픈 이라길래 6시 반에 갔는데 웨이 팅 왜케 많아 ㅠㅠ 오리 지날 양 꼬치 ...,오픈 반 웨이 팅 왜케 오리 양 꼬치 데리야끼 양 꼬치 바 로우 칭따오 맥주 여자 ...,6시 이라길래 6시 에 갔는데 많아 ㅠㅠ 지날 꿔 먹다가 배불러 죽는줄 알았다 그치...,"[꼬치, 묵음]",None,None,"[6시, 배불러, 죽는줄]","[꼬치, 묵음, 6시, 배불러, 죽는줄]"
96569,1068851079,정은,리지 널 양 꼬치 데리야끼 양 꼬치,리지 널 양 꼬치 데리야끼 양 꼬치,,"[꼬치, 데리야끼]",None,None,"[00, 000, 0000]","[꼬치, 데리야끼, 00, 000, 0000]"
96570,1068851079,파루,너무 양 냄새 없다는 장점 이자 단점 양 갈비 강추 익히는데 걸리지만 그만큼 맛있다...,양 냄새 장점 이자 단점 양 갈비 강추 우도,없다는 익히는데 걸리지만 그만큼 다 꿔 특별했다,"[이자, 우도]",맛있다,None,"[특별했다, 익히는데, 걸리지만]","[이자, 우도, 특별했다, 익히는데, 걸리지만]"
96571,1068851079,그리고,리지 널 양 갈비 180 g 13000원 순한 양 꼬치 180 g 12000원 가지...,리지 널 양 갈비 양 꼬치 가지 튀김 볶음 하얼빈 맥주 양 꼬치 해도 무방 정도 고...,180 g 13000원 순한 180 g 12000원 17000원 7000원 는 잡내...,"[가지, 튀김]",None,None,"[180, 부드러웠다, 고소한게]","[가지, 튀김, 180, 부드러웠다, 고소한게]"


In [26]:
review_df['keyword'] = review_df['keyword'].apply(lambda x: ' '.join(x))

In [27]:
review_df

,diner_idx,reviewer_id,notstopwords,nouns,others,n_keyword,good,bad,o_keyword,keyword
0,216642463,정희,반찬 가짓수 맛 은 평범할 수 있는데 정도 면 깨끗하고 규모 커서 혼밥 도 무리 없어요,반찬 가짓수 맛 은 수 정도 면 규모 커서 혼밥 도 무리,평범할 있는데 깨끗하고 없어요,"[무리, 규모]",None,None,"[평범할, 깨끗하고, 없어요]",무리 규모 평범할 깨끗하고 없어요
1,9346076,이지수,근처 운동 끝나고 서 제육 먹으면 그렇게 맛있을 수가 없습니다 반찬 밥 국 다 셀프...,근처 운동 제육 수가 반찬 밥 국 셀프 담 수 수 제육 고기 도 메뉴 도 예정,끝나고 서 먹으면 그렇게 을 없습니다 다 있어서 원하는만큼 먹을 있고 많이 주세요 ...,"[제육, 운동]",맛있다,None,"[원하는만큼, 보러, 끝나고]",제육 운동 원하는만큼 보러 끝나고
2,9346076,손석윤,일주일 한번 씩 은 꼭 먹는 코다리 찜 사장 님 직접 소스 비법 만들어서 먹고나면 ...,일주일 한번 은 꼭 코다리 찜 사장 직접 소스 비법 집 맛집 이기 집 코다리 이집 ...,씩 먹는 님 만들어서 먹고나면 생각나는 먹는내내 생각나는구나 는,"[코다리, 토욜]",맛있다,None,"[생각나는구나, 먹는내내, 먹고나면]",코다리 토욜 생각나는구나 먹는내내 먹고나면
3,26961859,세렌디피티,예전 맛있게 먹던 집 오랜 만 생각나서 갔더니 올 3월 더 영업 안 한다고 함,예전 집 만 더 영업 안 함,게 먹던 오랜 생각나서 갔더니 올 3월 한다고,"[영업, 예전]",맛있다,None,"[3월, 생각나서, 갔더니]",영업 예전 3월 생각나서 갔더니
4,26961859,고독한 탐식가,인당 일만원 이라는 어마어마한 가격 김치찌개 육질 좋은 삼겹살 사용 하고 정갈한 밑...,일만원 가격 김치찌개 육질 삼겹살 사용 밑반찬 도 가성,인당 이라는 어마어마한 좋은 하고 정갈한 좋았지만 비는 그저 그렇다,"[일만원, 육질]",None,None,"[어마어마한, 그렇다, 정갈한]",일만원 육질 어마어마한 그렇다 정갈한
...,...,...,...,...,...,...,...,...,...,...
96568,1068851079,블블,6시 오픈 이라길래 6시 반에 갔는데 웨이 팅 왜케 많아 ㅠㅠ 오리 지날 양 꼬치 ...,오픈 반 웨이 팅 왜케 오리 양 꼬치 데리야끼 양 꼬치 바 로우 칭따오 맥주 여자 ...,6시 이라길래 6시 에 갔는데 많아 ㅠㅠ 지날 꿔 먹다가 배불러 죽는줄 알았다 그치...,"[꼬치, 묵음]",None,None,"[6시, 배불러, 죽는줄]",꼬치 묵음 6시 배불러 죽는줄
96569,1068851079,정은,리지 널 양 꼬치 데리야끼 양 꼬치,리지 널 양 꼬치 데리야끼 양 꼬치,,"[꼬치, 데리야끼]",None,None,"[00, 000, 0000]",꼬치 데리야끼 00 000 0000
96570,1068851079,파루,너무 양 냄새 없다는 장점 이자 단점 양 갈비 강추 익히는데 걸리지만 그만큼 맛있다...,양 냄새 장점 이자 단점 양 갈비 강추 우도,없다는 익히는데 걸리지만 그만큼 다 꿔 특별했다,"[이자, 우도]",맛있다,None,"[특별했다, 익히는데, 걸리지만]",이자 우도 특별했다 익히는데 걸리지만
96571,1068851079,그리고,리지 널 양 갈비 180 g 13000원 순한 양 꼬치 180 g 12000원 가지...,리지 널 양 갈비 양 꼬치 가지 튀김 볶음 하얼빈 맥주 양 꼬치 해도 무방 정도 고...,180 g 13000원 순한 180 g 12000원 17000원 7000원 는 잡내...,"[가지, 튀김]",None,None,"[180, 부드러웠다, 고소한게]",가지 튀김 180 부드러웠다 고소한게


In [28]:
#'000'을 포함하지 않는 행만 필터링하여 review_df_에 저장

review_df = review_df[~review_df['keyword'].str.contains('000')]

In [29]:
review_df

,diner_idx,reviewer_id,notstopwords,nouns,others,n_keyword,good,bad,o_keyword,keyword
0,216642463,정희,반찬 가짓수 맛 은 평범할 수 있는데 정도 면 깨끗하고 규모 커서 혼밥 도 무리 없어요,반찬 가짓수 맛 은 수 정도 면 규모 커서 혼밥 도 무리,평범할 있는데 깨끗하고 없어요,"[무리, 규모]",None,None,"[평범할, 깨끗하고, 없어요]",무리 규모 평범할 깨끗하고 없어요
1,9346076,이지수,근처 운동 끝나고 서 제육 먹으면 그렇게 맛있을 수가 없습니다 반찬 밥 국 다 셀프...,근처 운동 제육 수가 반찬 밥 국 셀프 담 수 수 제육 고기 도 메뉴 도 예정,끝나고 서 먹으면 그렇게 을 없습니다 다 있어서 원하는만큼 먹을 있고 많이 주세요 ...,"[제육, 운동]",맛있다,None,"[원하는만큼, 보러, 끝나고]",제육 운동 원하는만큼 보러 끝나고
2,9346076,손석윤,일주일 한번 씩 은 꼭 먹는 코다리 찜 사장 님 직접 소스 비법 만들어서 먹고나면 ...,일주일 한번 은 꼭 코다리 찜 사장 직접 소스 비법 집 맛집 이기 집 코다리 이집 ...,씩 먹는 님 만들어서 먹고나면 생각나는 먹는내내 생각나는구나 는,"[코다리, 토욜]",맛있다,None,"[생각나는구나, 먹는내내, 먹고나면]",코다리 토욜 생각나는구나 먹는내내 먹고나면
3,26961859,세렌디피티,예전 맛있게 먹던 집 오랜 만 생각나서 갔더니 올 3월 더 영업 안 한다고 함,예전 집 만 더 영업 안 함,게 먹던 오랜 생각나서 갔더니 올 3월 한다고,"[영업, 예전]",맛있다,None,"[3월, 생각나서, 갔더니]",영업 예전 3월 생각나서 갔더니
4,26961859,고독한 탐식가,인당 일만원 이라는 어마어마한 가격 김치찌개 육질 좋은 삼겹살 사용 하고 정갈한 밑...,일만원 가격 김치찌개 육질 삼겹살 사용 밑반찬 도 가성,인당 이라는 어마어마한 좋은 하고 정갈한 좋았지만 비는 그저 그렇다,"[일만원, 육질]",None,None,"[어마어마한, 그렇다, 정갈한]",일만원 육질 어마어마한 그렇다 정갈한
...,...,...,...,...,...,...,...,...,...,...
96567,1068851079,블루스카이,맛 도 아주 좋고 친철 합니다,맛 도 아주 친철,좋고 합니다,"[친철, 아주]",None,None,"[합니다, 좋고, 00]",친철 아주 합니다 좋고 00
96568,1068851079,블블,6시 오픈 이라길래 6시 반에 갔는데 웨이 팅 왜케 많아 ㅠㅠ 오리 지날 양 꼬치 ...,오픈 반 웨이 팅 왜케 오리 양 꼬치 데리야끼 양 꼬치 바 로우 칭따오 맥주 여자 ...,6시 이라길래 6시 에 갔는데 많아 ㅠㅠ 지날 꿔 먹다가 배불러 죽는줄 알았다 그치...,"[꼬치, 묵음]",None,None,"[6시, 배불러, 죽는줄]",꼬치 묵음 6시 배불러 죽는줄
96570,1068851079,파루,너무 양 냄새 없다는 장점 이자 단점 양 갈비 강추 익히는데 걸리지만 그만큼 맛있다...,양 냄새 장점 이자 단점 양 갈비 강추 우도,없다는 익히는데 걸리지만 그만큼 다 꿔 특별했다,"[이자, 우도]",맛있다,None,"[특별했다, 익히는데, 걸리지만]",이자 우도 특별했다 익히는데 걸리지만
96571,1068851079,그리고,리지 널 양 갈비 180 g 13000원 순한 양 꼬치 180 g 12000원 가지...,리지 널 양 갈비 양 꼬치 가지 튀김 볶음 하얼빈 맥주 양 꼬치 해도 무방 정도 고...,180 g 13000원 순한 180 g 12000원 17000원 7000원 는 잡내...,"[가지, 튀김]",None,None,"[180, 부드러웠다, 고소한게]",가지 튀김 180 부드러웠다 고소한게


In [30]:
review_df.to_csv('review_keyword.csv', index=False)

In [1]:
# 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

tfidf_reviews = tfidf_vectorizer.fit_transform(review_df['notstopwords'])
tfidf_keyword = tfidf_vectorizer.transform(review_df['keyword'])
cosine_sim = cosine_similarity(tfidf_reviews, tfidf_keyword)

# 유사도 결과를 데이터프레임에 추가
review_df['cosine_sim'] = [cosine_sim[i, i] for i in range(cosine_sim.shape[0])]

NameError: name 'tfidf_vectorizer' is not defined

In [ ]:
review_df

In [ ]:
# 코사인 유사도 분포 시각화
plt.figure(figsize=(10, 6))  
sns.histplot(review_df['cosine_sim'], kde=True)  
plt.title('코사인 유사도 분포')  
plt.xlabel('코사인 유사도')  
plt.ylabel('빈도')  
plt.show() 

print(review_df['cosine_sim'].describe())  

# 코사인 유사도 0.3 이하의 리뷰 필터링
low_similarity_reviews = review_df[review_df['cosine_sim'] < 0.3]  
print("\n코사인 유사도가 낮은 리뷰:")
print(low_similarity_reviews[['notstopwords', 'keyword', 'cosine_sim']])  